In [1]:
import os.path

import boto3
import pandas as pd
from decouple import config
from s3logparse.s3logparse import parse_log_lines, LogLine

In [2]:
s3 = boto3.resource(
    's3',
    aws_access_key_id=config('ACCESS_KEY'),
    aws_secret_access_key=config('SECRET_KEY')
)

In [3]:
bucket = s3.Bucket(name='encode-public-logs')

In [4]:
LOG_PATH = 's3_logs'

if not os.path.exists(LOG_PATH):
    os.mkdir(LOG_PATH)

In [5]:
def get_local_or_download(name):
    local_path = f'{LOG_PATH}/{name}'
    if not os.path.exists(local_path):
        bucket.download_file(name, local_path)
    return open(local_path)

In [63]:
rows = []
column_headers = LogLine._fields

ALREADY_TRIED_FILE_PATH = 'already-tried.txt'

already_tried = []
if os.path.exists(ALREADY_TRIED_FILE_PATH):
    with open(ALREADY_TRIED_FILE_PATH, 'r') as already_tried_file:
        already_tried = already_tried_file.read().splitlines()

for file in bucket.objects.filter(Prefix='2019', Delimiter='-').limit(1500):
    if file.key in already_tried:
        continue
    local_path = f'{LOG_PATH}/{file.key}'
    if os.path.exists(local_path):
        with open(local_path, "r") as local_copy:
            content = local_copy.readlines()
    else:
        content = file.get()['Body'].read()
        if b'REST.GET.OBJECT' in content:
            with open(local_path, 'w') as local_copy:
                local_copy.write(str(content))
        else:
            already_tried.append(file.key)
            continue
    with get_local_or_download(file.key) as log:
        for log_entry in parse_log_lines(log.readlines()):
            rows.append(list(log_entry))
with open(ALREADY_TRIED_FILE_PATH, 'w') as already_tried_file:
    for tried in already_tried:
        already_tried_file.write(tried + '\n')

data = pd.DataFrame(rows, columns=column_headers)
print(data)

Empty DataFrame
Columns: [bucket_owner, bucket, timestamp, remote_ip, requester, request_id, operation, s3_key, request_uri, status_code, error_code, bytes_sent, object_size, total_time, turn_around_time, referrer, user_agent, version_id]
Index: []
